    작성일 : 20/01/03
    작성자 : 양희승
    작성내용 : 브랜드 목록 크롤링

    최종 : 2020-01-04

In [1]:
import requests
from bs4 import BeautifulSoup
import pymysql

In [2]:
#DB 연결 데이터 
config = {"host":"192.168.0.12", "user":"musinsa", "passwd":"1111", "db":"PR_TEST"}
conn = pymysql.connect(**config)

#html 파싱
def get_html(url):
    html = ""
    resp = requests.get(url)
    
    if resp.status_code == 200 :
        html = resp.text
        
    return html

In [3]:
#브랜드 목록, 링크 크롤링
def crawl_musinsa_brand() :
    brands_eng = []           #브랜드명(영문)을 담을 리스트
    brands_kor = []       #브랜드명(한글)을 담을 리스트
    brand_links = []      #브랜드 링크를 담을 리스트  
    brand_codes = []      #브랜드 코드를 담을 리스트
    cursor = conn.cursor()
    
    html = get_html("https://store.musinsa.com/app/contents/brandshop")
    soup = BeautifulSoup(html)
    

    for li in soup('li'):
        if('class' in li.attrs
            and 'brand_li' in li['class']) :
            brands_eng.append(li.dl.dt.a.get_text().strip())
            brands_kor.append(li.dl.dd.a.get_text().strip())
            brand_links.append("https://store.musinsa.com"+li.dl.dt.a['href'])
            brand_codes.append(li.dl.dt.a['href'].replace("/app/brand/goods_list/",""))

            
    
    #DB 적재
#     sql = ""
#     for i in range(len(brands_eng)) :
        
#         data = [str(i+1), brands_eng[i], brands_kor[i], brand_links[i], brand_codes[i]]
#         #sql = "INSERT INTO BRAND_LIST(BRD_NAME, BRD_LINK, BRD_CODE) VALUES("+ brands[i] + ", " + brand_links[i] + ", " + brand_codes[i] +")"
#         #cursor.execute(sql)
#         cursor.execute("insert into BRAND_LIST(BRD_NO, BRD_NAME_ENG, BRD_NAME_KOR, BRD_LINK, BRD_CODE) values(%s, %s, %s, %s, %s)", data)
#         conn.commit()
    
    
    
    
    #그냥 출력용
    for i in range(len(brands_eng)) :
        print("No." + str(i+1) + "\n"
              + "브랜드명(영문) : " + brands_eng[i] + "\n"
              + "브랜드명(한글) : " + brands_kor[i] + "\n"
              + "브랜드 링크 : " + brand_links[i] + "\n"
              + "브랜드 코드 : " + brand_codes[i] + "\n"
             )

#     #CSV파일로 저장
#     file = csv.writer(open("data/brand.csv", "w"), delimiter = ",")
#     header = ["No", "BRD_NAME_ENG", "BRD_NAME_KOR", "BRD_LINK", "BRD_CODE"]  
#     file.writerow(header)
#     brd_tuple = ()
#     for i in range(len(brands)) :
#         brd_tuple = (str(i+1), brands_eng[i], brands_kor[i], brand_links[i], brand_codes[i])
#         file.writerow(brd_tuple)